# 1. 그룹바이 심화

# 1. 일반 분석 (기본 접근방법)

In [1]:
import pandas as pd

selloutData = pd.read_csv("../dataset/kopo_product_volume.csv")
selloutData.head()

selloutData.columns = selloutData.columns.str.lower()

sortedData = selloutData.sort_values(["regionid","productgroup","yearweek"])
sortedData.reset_index(inplace=True, drop=True)
sortedData.head()

sortedData["ma"] = sortedData["volume"].rolling(window=17, center=False, min_periods=1).mean()
sortedData.head()

,regionid,productgroup,yearweek,volume,ma
0,A01,ST0001,201401,513598,513598.000000
1,A01,ST0001,201402,438251,475924.500000
2,A01,ST0001,201403,420290,457379.666667
3,A01,ST0001,201404,458431,457642.500000
4,A01,ST0001,201405,482381,462590.200000


In [2]:
sortedData.to_csv("./group_test.csv")

# 2. 집계 분석 (1단계 업 분석방법)

In [3]:
import pandas as pd

selloutData = pd.read_csv("../dataset/kopo_product_volume.csv")
selloutData.head()

selloutData.columns = selloutData.columns.str.lower()

sortedData = selloutData.sort_values(["regionid","productgroup","yearweek"])
sortedData.reset_index(inplace=True, drop=True)
sortedData.head()

groupResult = sortedData.groupby(["regionid","productgroup"])["volume"].mean()
groupResult = groupResult.reset_index()
groupResult.rename(columns={'volume': 'avg_volume'}, inplace=True)
groupResult.head(2)

,regionid,productgroup,avg_volume
0,A01,ST0001,672850.421053
1,A01,ST0002,189489.924242


In [4]:
finalResult = sortedData.merge(groupResult, how="left", 
                 on = ["regionid","productgroup"])
finalResult.head()

,regionid,productgroup,yearweek,volume,avg_volume
0,A01,ST0001,201401,513598,672850.421053
1,A01,ST0001,201402,438251,672850.421053
2,A01,ST0001,201403,420290,672850.421053
3,A01,ST0001,201404,458431,672850.421053
4,A01,ST0001,201405,482381,672850.421053


In [5]:
finalResult.to_csv("./groupResult.csv")

# 3. 집계 분석 (고급 분석방법)

In [15]:
def sub_function(data):
    data["ma"] = data["qty"].rolling(window = 5, center = True, min_periods=1).mean()
    return data

In [16]:
import pandas as pd

selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0


In [17]:
selloutData.columns = selloutData.columns.str.lower()
selloutData.head()

,regionid,product,yearweek,qty
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0


In [18]:
sortedData = selloutData.sort_values(["regionid","product","yearweek"])
sortedData.head()

,regionid,product,yearweek,qty
298,A00,PRODUCT34,201401,661.0
1757,A00,PRODUCT34,201402,679.0
3125,A00,PRODUCT34,201403,578.0
205,A00,PRODUCT34,201404,532.0
4369,A00,PRODUCT34,201405,516.0


In [19]:
sortedData.reset_index(inplace=True, drop=True)
sortedData.head()

,regionid,product,yearweek,qty
0,A00,PRODUCT34,201401,661.0
1,A00,PRODUCT34,201402,679.0
2,A00,PRODUCT34,201403,578.0
3,A00,PRODUCT34,201404,532.0
4,A00,PRODUCT34,201405,516.0


In [25]:
finalGroupResult = sortedData.groupby(["regionid","product"]).apply(sub_function)

In [27]:
finalGroupResult.head()

,regionid,product,yearweek,qty,ma
0,A00,PRODUCT34,201401,661.0,639.333333
1,A00,PRODUCT34,201402,679.0,612.500000
2,A00,PRODUCT34,201403,578.0,593.200000
3,A00,PRODUCT34,201404,532.0,545.600000
4,A00,PRODUCT34,201405,516.0,491.200000


### 디버깅 방법

In [28]:
groupData = sortedData.groupby(["regionid","product"])

In [29]:
len(groupData)

794

In [30]:
eachData = groupData.get_group(list(groupData.groups)[793])
eachData.head()

,regionid,product,yearweek,qty
124501,A77,PRODUCT12,201401,3821.0
124502,A77,PRODUCT12,201402,2466.0
124503,A77,PRODUCT12,201403,2401.0
124504,A77,PRODUCT12,201404,2628.0
124505,A77,PRODUCT12,201405,1442.0


In [31]:
eachData = groupData.get_group(list(groupData.groups)[793])
eachData.head()

,regionid,product,yearweek,qty
124501,A77,PRODUCT12,201401,3821.0
124502,A77,PRODUCT12,201402,2466.0
124503,A77,PRODUCT12,201403,2401.0
124504,A77,PRODUCT12,201404,2628.0
124505,A77,PRODUCT12,201405,1442.0


### [실습]
kopo_channel_seasonality 데이터를 활용하여
이동편균구간 17로 (center)
이동평균 값을 (regionid,product) 카테고리별로
산출하여 “ma” 컬럼을 생성하세요

In [45]:
period = 17

In [46]:
def ma_rolling(eachData, term):
    basicData = eachData.reset_index(drop=True)
    #basicData.head(2)

    basicData["ma"] = basicData["qty"].rolling(window=term, center=True, min_periods=1).mean()
    #basicData.head(2)
    return basicData

In [47]:
groupResult = sortedData.groupby(["regionid","product"]).apply(ma_rolling, period)

In [48]:
groupResult.head(2)

regionid    product  yearweek    qty          ma
regionid product                                                     
A00      PRODUCT34 0      A00  PRODUCT34    201401  661.0  514.444444
                   1      A00  PRODUCT34    201402  679.0  516.800000

In [49]:
groupResultMa = groupResult.reset_index(drop=True)

In [50]:
groupResultMa.head()

,regionid,product,yearweek,qty,ma
0,A00,PRODUCT34,201401,661.0,514.444444
1,A00,PRODUCT34,201402,679.0,516.800000
2,A00,PRODUCT34,201403,578.0,503.363636
3,A00,PRODUCT34,201404,532.0,490.750000
4,A00,PRODUCT34,201405,516.0,480.230769


### [실습]
kopo_channel_seasonality 데이터를 활용하여
이동편균구간 9로 (center)
이동표준편차 값을 (regionid,product) 카테고리별로
산출하여 “stdev” 컬럼을 생성하세요


In [39]:
periodStd = 9

In [42]:
def std_rolling(eachData, term):
    basicData = eachData.reset_index(drop=True)
    #basicData.head(2)

    basicData["stdev"] = basicData["qty"].rolling(window=term, center=True, min_periods=1).std()
    #basicData.head(2)
    return basicData

In [43]:
groupResultStd = sortedData.groupby(["regionid","product"]).apply(std_rolling, periodStd)

In [44]:
groupResultStd.head(2)

regionid    product  yearweek    qty      stdev
regionid product                                                    
A00      PRODUCT34 0      A00  PRODUCT34    201401  661.0  73.984458
                   1      A00  PRODUCT34    201402  679.0  95.952940